# 초기화

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableBranch

from langchain.memory import ConversationBufferMemory
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

from operator import itemgetter

In [ ]:
# 프롬프트 분류기 

from typing import Literal
from langchain.pydantic_v1 import BaseModel
from langchain.output_parsers.openai_functions import PydanticAttrOutputFunctionsParser
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

In [ ]:
# output parser 

from typing import List

from langchain.llms import OpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain.pydantic_v1 import BaseModel, Field, validator

In [ ]:
from langchain.schema.runnable import RunnableParallel

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv

print(load_dotenv(find_dotenv(), override=True))

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

True


In [ ]:
memory = ConversationBufferMemory(return_messages=True)

# DB 생성 및 CURD method 구현

In [ ]:
from sqlalchemy import create_engine, Column, String, Integer, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

# 데이터베이스 설정
engine = create_engine('sqlite:///your_database.db')  # 여기서는 SQLite를 예시로 사용합니다.
Base = declarative_base()

class Product(Base):
    __tablename__ = 'products'

    id = Column(Integer, primary_key=True, autoincrement=True)
    user_id = Column(String, index=True)
    order_id = Column(Integer, index=True)  # 필요에 따라 유지하거나 제거
    datetime = Column(String, index=True)  # 문자열 타입으로 datetime 필드 추가
    name = Column(String(50), nullable=False)
    quantity = Column(Integer)
    price = Column(Integer)
    total_price = Column(Integer)
    
    
# 테이블 생성
Base.metadata.create_all(engine)

# 세션 생성을 위한 설정
Session = sessionmaker(bind=engine)

C:\Users\ktsfr\AppData\Local\Temp\ipykernel_13544\3616412217.py:7: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [ ]:
from pydantic import BaseModel
from typing import List, Dict
from datetime import datetime

# Pydantic 모델 정의
class Order(BaseModel):
    datetime: str
    products: List[Dict[str, Dict[str, int]]]

# 데이터 생성 (Create)
def create_order(order_dict):
    session = Session()
    for product_info in order_dict['products']:
        for product_name, details in product_info.items():
            new_product = Product(
                user_id=order_dict['user_id'],
                datetime=order_dict['datetime'],  # datetime 필드 추가
                name=product_name,
                quantity=details['수량'],
                price=details['가격'],
                total_price=details['수량'] * details['가격']
            )
            session.add(new_product)
    session.commit()
    return "주문이 성공적으로 저장되었습니다."

# 데이터 조회 (Read)
def get_order(order: Order):
    session = Session()
    return session.query(Product).filter(Product.datetime == order.datetime).all()

# 데이터 업데이트 (Update)
def update_order(order: Order):
    session = Session()
    products = session.query(Product).filter(Product.datetime == order.datetime).all()
    if products:
        for product in products:
            for product_info in order.products:
                if product.name in product_info:
                    product.quantity = product_info[product.name]['수량']
                    product.price = product_info[product.name]['가격']
                    product.total_price = product_info[product.name]['수량'] * product_info[product.name]['가격']
        session.commit()
    return "주문이 성공적으로 변경되었습니다."

# 데이터 삭제 (Delete)
def delete_order(order: Order):
    session = Session()
    products = session.query(Product).filter(Product.datetime == order.datetime).all()
    for product in products:
        session.delete(product)
    session.commit()
    return "주문이 성공적으로 취소되었습니다."

# 작업 chain 구성
- new message
    - chat_status_chain
    - message_classifier_chain(주문, 주문 조회 등인지 분류)
        - 대화 종료 -> say_goodbye_chain
        - 대화 중 & 일반 대화 -> response_chain
        - 대화 중 & 작업 요청 -> handle_request_chain 
---
- 가능 대안
- new message
    - chat_status_chain
    - message_classifier_chain(주문 문의, 주문 요청, 주문 변경 문의, 주문변경 처리 등 분류)
        - 대화 종료 -> say_goodbye_chain
        - 대화 중 -> 해당 작업 chain

### run() 

In [ ]:
def handle_chain(new_message, memory):
    chat_history = memory.load_memory_variables()["history"]
    status = chat_status_chain.invoke({"customer_message": new_message, "history": chat_history})
    if status["conv_status"] == "대화 중":
        result = parallel_processing.invoke({"customer_message": new_message, 'history': chat_history}) # result: {'response', 'handle_request'}
    else:
        result = chat_end_chain.invoke({"customer_message": new_message, 'history': chat_history})
    return result

### chain structure (대화 종료가 아닌 일반적인 경우에 사용하는 체인들의 구조)

In [ ]:
chat_chain = intention_classifier_chain | chat_type_branch

In [ ]:
intention_classifier_chain = (
    RunnablePassthrough.assign(intention=( intention_classifier_prompt | ChatOpenAI(model="gpt-3.5-turbo-0613") | StrOutputParser()) )
)

In [ ]:
chat_type_branch = RunnableBranch(
  (lambda x: x["intention"] == "일반 대화", response_chain),
  handle_request_chain
)

In [ ]:
inquiry_classifier_chain = inquiry_classifier_prompt | ChatOpenAI(model="gpt-3.5-turbo-0613") | StrOutputParser()

inquiry_type_branch = RunnableBranch(
  (lambda x: x["inquiry"] == "주문 변경", change_inquiry_chain),
  (lambda x: x["inquiry"] == "주문 취소", cancel_inquiry_chain),
  general_inquiry_chain
)

general_inquiry_chain = general_prompt | ChatOpenAI(model="gpt-3.5-turbo-0613") | StrOutputParser()

response_chain = RunnablePassthrough.assign(inquiey=inquiry_classifier_chain) | inquiry_type_branch 

In [ ]:
request_classifier_chain = RunnablePassthrough.assign(request=request_classifier_prompt | ChatOpenAI(model="gpt-3.5-turbo-0613") | StrOutputParser() )

order_chain = (
    RunnablePassthrough.assign(parsed_record=order_prompt | ChatOpenAI(model="gpt-3.5-turbo-0613") | order_record_parser) |
    RunnablePassthrough.assign(task_result=RunnableLambda(process_order)) |
    report_chain
)
report_chain = report_prompt | ChatOpenAI(model="gpt-3.5-turbo-0613") | StrOutputParser()

request_type_branch = RunnableBranch(
  (lambda x: x["request"] == "주문 조회 요청", order_query_chain),
  (lambda x: x["request"] == "주문 변경 요청", order_change_chain),
  (lambda x: x["request"] == "주문 취소 요청", order_cancel_chain),
  order_chain
)

handle_chain = (
    RunnablePassthrough.assign(request_classifier_chain) |
    request_type_branch
)

### 공통 사용 함수

In [3]:
def save_conversation(dict):
    memory.save_context({"inputs": dict["message"]}, {"output": dict["response"]})
    return dict

### prompt

In [ ]:
general_prompt = PromptTemplate.from_template("""
너는 고객 문의를 매우 많이 해본 뛰어난 주문봇이야.
가게에서 판매하는 상품 정보를 바탕으로 고객 문의에 친절하고 자세하게 답변해줘.

<이전 대화 내용>을 고려해서 답변해야 해. 
<이전 대화 내용>에서 'HumanMessage'는 고객이 말한 내용이고, 'BotMessage'는 너가 말한 내용이야.

자연스럽게 주문으로 이어지도록 대화를 이어가되, 지나치게 주문을 유도하지는 말아줘.

주문을 파악할 때는 다음 순서대로 진행해줘.
1. 고객이 언급한 상품과 가장 비슷한 상품을 상품 목록에서 찾기.
2. 고객이 언급한 상품 수량은 상품 목록의 '기본 판매 수량'을 기준으로 해석하기.
3. 고객 주문은 상품명, 주문 수량, 총 주문 가격으로 파악하기.

<가게에서 판매하는 상품 목록>
1. 상품명: 떡케익5호
   기본 판매 수량: 1개
   기본 판매 수량의 가격: 54,000원
2. 상품명: 무지개 백설기 케익
   기본 판매 수량: 1개
   기본 판매 수량의 가격: 51,500원
3. 상품명: 미니 백설기
   기본 판매 수량: 35개
   기본 판매 수량의 가격: 31,500원
4. 상품명: 개별 모듬팩
   기본 판매 수량: 1개
   기본 판매 수량의 가격: 13,500원
   

<이전 대화 내용>은 다음과 같아:
{history}

<고객 문의>는 다음과 같아:
{customer_message}
답변:""")

order_query_prompt = PromptTemplate.from_template("""
너는 주문 내역 조회를 매우 많이 해본 뛰어난 주문봇이야.
가게에서 판매하는 상품 정보를 바탕으로 고객 문의에 친절하고 자세하게 답변해줘.

<이전 대화 내용>을 고려해서 답변해야 해. 
<이전 대화 내용>에서 'HumanMessage'는 고객이 말한 내용이고, 'BotMessage'는 너가 말한 내용이야.

<가게에서 판매하는 상품 목록>
1. 상품: 떡케익5호
   기본 판매 수량: 1개
   기본 판매 수량의 가격: 54,000원
2. 상품: 무지개 백설기 케익
   기본 판매 수량: 1개
   기본 판매 수량의 가격: 51,500원
3. 상품: 미니 백설기
   기본 판매 수량: 35개
   기본 판매 수량의 가격: 31,500원
4. 상품: 개별 모듬팩
   기본 판매 수량: 1개
   기본 판매 수량의 가격: 13,500원

<이전 대화>
{history}

<고객 문의>는 다음과 같아:
{customer_message}
답변:""")

order_change_prompt = PromptTemplate.from_template("""
너는 주문 변경을 전담하는 뛰어난 주문봇이야.
<이전 대화 내용>을 고려해서 답변해야 해.
<이전 대화 내용>에서 'HumanMessage'는 고객이 말한 내용이고, 'BotMessage'는 너가 말한 내용이야.

작업 진행 원칙
- 고객이 변경한 주문 내용을 정확하게 파악하고, 너가 파악한 내용이 맞는지 고객에게 한 번 더 확인해줘.
- 너가 파악한 주문 변경 내용이 잘못됐다면, 주문 변경 내용을 정확히 파악하고 그 내용이 맞는지 고객에게 확인하는 작업을 주문 변경 내용을 정확히 파악할 때까지 반복해야돼.
- 고객의 주문 변경을 정확히 파악했다면, 고객에게 고객이 주문을 변경한 상품의 이름, 수량, 가격을 각각 알려주고, 마지막에는 변경된 주문의 총 가격을 알려줘.


<가게에서 판매하는 상품 목록>
1. 상품명: 떡케익5호
   기본 판매 수량: 1개
   기본 판매 수량의 가격: 54,000원
2. 상품명: 무지개 백설기 케익
   기본 판매 수량: 1개
   기본 판매 수량의 가격: 51,500원
3. 상품명: 미니 백설기
   기본 판매 수량: 35개
   기본 판매 수량의 가격: 31,500원
4. 상품명: 개별 모듬팩
   기본 판매 수량: 1개
   기본 판매 수량의 가격: 13,500원

<이전 대화 내용>은 다음과 같아:
{history}


<고객의 주문 변경>은 다음과 같아:
{customer_message}
답변:""")

order_cancel_prompt = PromptTemplate.from_template("""
너는 주문 취소를 전담하는 주문봇이야.
<이전 대화 내용>을 고려해서 답변해야 해.
<이전 대화 내용>에서 'HumanMessage'는 고객이 말한 내용이고, 'BotMessage'는 너가 말한 내용이야.

작업 진행 원칙
- 고객이 취소하려는 주문을 정확하게 파악하고, 너가 파악한 내용이 맞는지 고객에게 한 번 더 확인해줘.
- 너가 파악한 주문 취소 내용이 잘못됐다면, 주문 취소 내용을 정확히 파악하고 그 내용이 맞는지 고객에게 확인하는 작업을 주문 취소 내용을 정확히 파악할 때
- 고객의 주문 취소 내용을 정확히 파악했다면, 고객에게 고객이 주문을 취소한 상품의 이름, 수량, 가격을 각각 알려주고, 마지막에는 취소된 주문의 총 가격을 알려줘.

<가게에서 판매하는 상품 목록>
1. 상품명: 떡케익5호
   기본 판매 수량: 1개
   기본 판매 수량의 가격: 54,000원
2. 상품명: 무지개 백설기 케익
   기본 판매 수량: 1개
   기본 판매 수량의 가격: 51,500원
3. 상품명: 미니 백설기
   기본 판매 수량: 35개
   기본 판매 수량의 가격: 31,500원
4. 상품명: 개별 모듬팩
   기본 판매 수량: 1개
   기본 판매 수량의 가격: 13,500원

<이전 대화 내용>은 다음과 같아:
{history}

<고객이 취소하려는 주문>은 다음과 같아:
{customer_message}
답변:""")

### chat_status_chain

In [ ]:
chat_status_prompt = PromptTemplate.from_template("""
너는 대화가 끝난 것 같은지, 아니면 계속 진행될 것 같은지를 예리하게 분별할 수 있는 대화 분류봇이야.
<이전 대화>와 <현재 고객이 입력한 메시지>를 모두 고려해서 대화가 끝난 것 같으면 '대화 종료'로, 대화가 계속 진행될 것 같으면 '대화 중'으로 분류해줘.
<이전 대화>에서 'HumanMessage'는 고객이 말한 내용이고, 'AIMessage'는 너가 말한 내용이야.

<이전 대화 내용>:
{history}

<현재 고객이 입력한 메시지>:
{customer_message}
분류 결과:""")

chat_status_classifier = chat_status_prompt | ChatOpenAI(model="gpt-3.5-turbo-0613") | StrOutputParser()
chat_status_chain = (
    RunnablePassthrough.assign(history=RunnableLambda(memory.load_memory_variables) | itemgetter("history")) |
    RunnablePassthrough.assign(conv_status=chat_status_classifier | StrOutputParser())
) 
chat_status_chain

### chat_end_chain

In [ ]:
chat_end_chain = PromptTemplate.from_template("""
아래 순서에 따라 대화를 종료해줘.
1. "<세상의 모든 떡>을 이용해주셔서 감사합니다 ^^"라고 감사 인사를 해줘.
2. <이전 대화>와 <가게에서 판매하는 상품 목록>를 보고 고객 문의에 대한 처리 결과를 ```로 감싸진 예시 형식으로 고객에게 알려줘.

```예시 형식
<고객이 주문을 한 경우>
주문 내역: 떡케익5호(54,000원) 1개, 무지개 백설기 케익(51,500원) 1개, 미니 백설기(31,500) 2개, 개별 모듬팩(13,500원) 1개, 총 주문 가격: 150,500원

<고객이 주문을 조회한 경우>
조회를 요청하신 주문 내역: 주문 내역: 떡케익5호(54,000원) 1개, 무지개 백설기 케익(51,500원) 1개, 미니 백설기(31,500) 2개, 개별 모듬팩(13,500원) 1개, 총 주문 가격: 150,500원

<고객이 주문을 변경한 경우>
변경된 주문 내역: 떡케익5호(54,000원) 1개, 무지개 백설기 케익(51,500원) 1개, 미니 백설기(31,500) 2개, 개별 모듬팩(13,500원) 1개, 총 주문 가격: 150,500원

<고객이 주문을 취소한 경우>
취소하신 주문 내역: 떡케익5호(54,000원) 1개, 무지개 백설기 케익(51,500원) 1개, 미니 백설기(31,500) 2개, 개별 모듬팩(13,500원) 1개, 총 주문 가격: 150,500원
```

가게에서 판매하는 상품 목록.
1. 상품: 떡케익5호
   기본 판매 수량: 1개
   기본 판매 수량의 가격: 54,000원
2. 상품: 무지개 백설기 케익
   기본 판매 수량: 1개
   기본 판매 수량의 가격: 51,500원
3. 상품: 미니 백설기
   기본 판매 수량: 35개
   기본 판매 수량의 가격: 31,500원
4. 상품: 개별 모듬팩
   기본 판매 수량: 1개
   기본 판매 수량의 가격: 13,500원
   
<이전 대화>
{history}

<고객이 현재 입력한 메시지>
{customer_message}
답변:""")

chat_end_chain = (
    RunnablePassthrough.assign(response=( chat_end_prompt | ChatOpenAI(model="gpt-3.5-turbo-0613") | StrOutputParser()) ) |
    RunnableLambda(save_conversation)
)

# say_goodbye_chain.invoke({"history": "안녕하세요.", "message": "또 인사 드려요."})

### chat_chain

In [ ]:
chat_chain = intention_classifier_chain | chat_type_branch

#### intention_classifier_chain (주문 문의인지 주문 요청인지, 주문 변경 문의인지 주문 변경 요청 등인지 구분하는 체인)

In [ ]:
intention_classifier_prompt = PromptTemplate.from_template("""

<이전 대화>
{history}

<고객이 현재 입력한 메시지>
{customer_message}
답변:""")

intention_classifier_chain = (
    RunnablePassthrough.assign(intention=( intention_classifier_prompt | ChatOpenAI(model="gpt-3.5-turbo-0613") | StrOutputParser()) )
)

#### router(chat_type_branch) - general chat || request chat 따라 해당되늰 chain으로 분기

In [ ]:
chat_type_branch = RunnableBranch(
  (lambda x: x["intention"] == "일반 대화", response_chain),
  handle_request_chain
)

#### response_chain

In [ ]:
response_chain = RunnablePassthrough.assign(inquiey=inquiry_classifier_chain) | inquiry_type_branch 
response_chain.invoke({'customer_message': '주문 변경을 해주세요.', 'history': ''})

##### inquiry_classifier_chain

In [ ]:
inquiry_classifier_prompt = PromptTemplate.from_template("""
너는 고객 문의 유형을 정확하게 분별할 수 있는 대화 분류봇이야.
<이전 대화>와 <현재 고객이 입력한 메시지>를 모두 고려해서 사용자 문의의 유형를 분류해줘.
"사용자 문의의 유형은 '일반', '주문 조회', '주문 변경', '주문 취소' 중 하나야. '일반'은 상품에 관한 질문과 상품 주문 모두를 포함해."
<이전 대화>에서 'HumanMessage'는 고객이 말한 내용이고, 'AIMessage'는 너가 말한 내용이야.

<이전 대화>:
{history}

<현재 고객이 입력한 메시지>:
{customer_message}
분류 결과:""")

inquiry_classifier_chain = inquiry_classifier_prompt | ChatOpenAI(model="gpt-3.5-turbo-0613") | StrOutputParser()

##### prompt

In [ ]:
general_prompt = PromptTemplate.from_template("""
너는 고객 문의를 매우 많이 해본 뛰어난 주문봇이야.
가게에서 판매하는 상품 정보를 바탕으로 고객 문의에 친절하고 자세하게 답변해줘.
<이전 대화 내용>을 고려해서 답변해야 해. 
<이전 대화 내용>에서 'HumanMessage'는 고객이 말한 내용이고, 'BotMessage'는 너가 말한 내용이야.
자연스럽게 주문으로 이어지도록 대화를 이어가되, 지나치게 주문을 유도하지는 말아줘.

주문을 파악할 때는 다음 순서대로 진행해줘.
1. 고객이 언급한 상품과 가장 비슷한 상품을 상품 목록에서 찾기.
2. 고객이 언급한 상품 수량은 상품 목록의 '기본 판매 수량'을 기준으로 해석하기.
3. 고객 주문은 상품명, 주문 수량, 총 주문 가격으로 파악하기.

<가게에서 판매하는 상품 목록>
1. 상품명: 떡케익5호
   기본 판매 수량: 1개
   기본 판매 수량의 가격: 54,000원
2. 상품명: 무지개 백설기 케익
   기본 판매 수량: 1개
   기본 판매 수량의 가격: 51,500원
3. 상품명: 미니 백설기
   기본 판매 수량: 35개
   기본 판매 수량의 가격: 31,500원
4. 상품명: 개별 모듬팩
   기본 판매 수량: 1개
   기본 판매 수량의 가격: 13,500원
   

<이전 대화 내용>은 다음과 같아:
{history}

<고객 문의>는 다음과 같아:
{customer_message}
답변:""")

order_change_prompt = PromptTemplate.from_template("""
너는 주문 변경을 전담하는 뛰어난 주문봇이야.
<이전 대화 내용>을 고려해서 답변해야 해.
<이전 대화 내용>에서 'HumanMessage'는 고객이 말한 내용이고, 'BotMessage'는 너가 말한 내용이야.

작업 진행 원칙
- 고객이 변경한 주문 내용을 정확하게 파악하고, 너가 파악한 내용이 맞는지 고객에게 한 번 더 확인해줘.
- 너가 파악한 주문 변경 내용이 잘못됐다면, 주문 변경 내용을 정확히 파악하고 그 내용이 맞는지 고객에게 확인하는 작업을 주문 변경 내용을 정확히 파악할 때까지 반복해야돼.
- 고객의 주문 변경을 정확히 파악했다면, 고객에게 고객이 주문을 변경한 상품의 이름, 수량, 가격을 각각 알려주고, 마지막에는 변경된 주문의 총 가격을 알려줘.


<가게에서 판매하는 상품 목록>
1. 상품명: 떡케익5호
   기본 판매 수량: 1개
   기본 판매 수량의 가격: 54,000원
2. 상품명: 무지개 백설기 케익
   기본 판매 수량: 1개
   기본 판매 수량의 가격: 51,500원
3. 상품명: 미니 백설기
   기본 판매 수량: 35개
   기본 판매 수량의 가격: 31,500원
4. 상품명: 개별 모듬팩
   기본 판매 수량: 1개
   기본 판매 수량의 가격: 13,500원

<이전 대화 내용>은 다음과 같아:
{history}


<고객의 주문 변경>은 다음과 같아:
{customer_message}
답변:""")

order_cancel_prompt = PromptTemplate.from_template("""
너는 주문 취소를 전담하는 주문봇이야.
<이전 대화 내용>을 고려해서 답변해야 해.
<이전 대화 내용>에서 'HumanMessage'는 고객이 말한 내용이고, 'BotMessage'는 너가 말한 내용이야.

작업 진행 원칙
- 고객이 취소하려는 주문을 정확하게 파악하고, 너가 파악한 내용이 맞는지 고객에게 한 번 더 확인해줘.
- 너가 파악한 주문 취소 내용이 잘못됐다면, 주문 취소 내용을 정확히 파악하고 그 내용이 맞는지 고객에게 확인하는 작업을 주문 취소 내용을 정확히 파악할 때
- 고객의 주문 취소 내용을 정확히 파악했다면, 고객에게 고객이 주문을 취소한 상품의 이름, 수량, 가격을 각각 알려주고, 마지막에는 취소된 주문의 총 가격을 알려줘.

<가게에서 판매하는 상품 목록>
1. 상품명: 떡케익5호
   기본 판매 수량: 1개
   기본 판매 수량의 가격: 54,000원
2. 상품명: 무지개 백설기 케익
   기본 판매 수량: 1개
   기본 판매 수량의 가격: 51,500원
3. 상품명: 미니 백설기
   기본 판매 수량: 35개
   기본 판매 수량의 가격: 31,500원
4. 상품명: 개별 모듬팩
   기본 판매 수량: 1개
   기본 판매 수량의 가격: 13,500원

<이전 대화 내용>은 다음과 같아:
{history}

<고객이 취소하려는 주문>은 다음과 같아:
{customer_message}
답변:""")

##### inquiry 별 chain

In [ ]:
general_inquiry_chain = general_prompt | ChatOpenAI(model="gpt-3.5-turbo-0613") | StrOutputParser()
change_inquiry_chain = order_change_prompt | ChatOpenAI(model="gpt-3.5-turbo-0613") | StrOutputParser()
cancel_inquiry_chain = order_cancel_prompt | ChatOpenAI(model="gpt-3.5-turbo-0613") | StrOutputParser()

##### router(inquiry_type_branch)

In [ ]:
inquiry_type_branch = RunnableBranch(
  (lambda x: x["inquiry"] == "주문 변경", change_inquiry_chain),
  (lambda x: x["inquiry"] == "주문 취소", cancel_inquiry_chain),
  general_chain
)
inquiry_type_branch

RunnableBranch(branches=[(RunnableLambda(...), PromptTemplate(input_variables=['customer_message', 'history'], template="\n너는 주문 변경을 전담하는 뛰어난 주문봇이야.\n<이전 대화 내용>을 고려해서 답변해야 해.\n<이전 대화 내용>에서 'HumanMessage'는 고객이 말한 내용이고, 'BotMessage'는 너가 말한 내용이야.\n\n작업 진행 원칙\n- 고객이 변경한 주문 내용을 정확하게 파악하고, 너가 파악한 내용이 맞는지 고객에게 한 번 더 확인해줘.\n- 너가 파악한 주문 변경 내용이 잘못됐다면, 주문 변경 내용을 정확히 파악하고 그 내용이 맞는지 고객에게 확인하는 작업을 주문 변경 내용을 정확히 파악할 때까지 반복해야돼.\n- 고객의 주문 변경을 정확히 파악했다면, 고객에게 고객이 주문을 변경한 상품의 이름, 수량, 가격을 각각 알려주고, 마지막에는 변경된 주문의 총 가격을 알려줘.\n\n\n<가게에서 판매하는 상품 목록>\n1. 상품명: 떡케익5호\n   기본 판매 수량: 1개\n   기본 판매 수량의 가격: 54,000원\n2. 상품명: 무지개 백설기 케익\n   기본 판매 수량: 1개\n   기본 판매 수량의 가격: 51,500원\n3. 상품명: 미니 백설기\n   기본 판매 수량: 35개\n   기본 판매 수량의 가격: 31,500원\n4. 상품명: 개별 모듬팩\n   기본 판매 수량: 1개\n   기본 판매 수량의 가격: 13,500원\n\n<이전 대화 내용>은 다음과 같아:\n{history}\n\n\n<고객의 주문 변경>은 다음과 같아:\n{customer_message}\n답변:")
| ChatOpenAI(client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo-0613', openai_api_key='sk-1UEI5br

#### handle_request_chain

In [ ]:
handle_chain = (
    RunnablePassthrough.assign(request_classifier_chain) |
    request_type_branch
)

##### request_classifier_chain

In [ ]:
request_classifier_prompt = PromptTemplate.from_template("""
너는 고객이 이 작업 중 하나를 구체적으로 요청하고 있는지를 정확하게 판단할 수 있는 로봇이야.

아래 순서에 따라 분류를 진행해줘.
고객이 구체적인 작업을 요청 중인지 아닌지, 만약 구체적인 작업을 요청 중이라면 어떤 종류의 작업을 요청 중인지를 정확하게 판단해줘.
1. <이전 대화>를 바탕으로 <고객이 현재 입력한 메시지>의 의미를 명확히 파악하기.
2. 현재 고객이 구체적인 작업을 요청 중인지, 단순 응답이나 질문을 하는 중인지 분류하기. 단순 응답이나 질문을 하는 중이라면 '없음'으로 분류하기.
2. 만약 현재 고객이 구체적인 작업을 요청 중이라면, '주문 요청', '주문 조회 요청', '주문 변경 요청', '주문 취소 요청' 중 어떤 종류의 작업을 요청 중인지 분류하기.
3. 분류 결과가 '요청 없음', '요청 주문', '주문 조회 요청', '주문 변경 요청', '주문 취소 요청' 중 하나가 아니라면 지금까지의 과정을 반복하여 정해진 분류 유형 중 하나로 반환하기.

분류 결과는 '요청 없음', '주문 요청', '주문 조회 요청', '주문 변경 요청', '주문 취소 요청' 중 하나여야만 헤.

<이전 대화>
{history}
<현재 고객이 입력한 메시지>:
{customer_message}
분류 결과:""")

request_classifier_chain = RunnablePassthrough.assign(request=request_classifier_prompt | ChatOpenAI(model="gpt-3.5-turbo-0613") | StrOutputParser() )
request_classifier_chain.invoke({"customer_message": "네 ", 'history': ['떡케익1, 미니백설기2, 개별모듬팩3 주문할게요',
                                                                '고객님께서는 떡케익 1개, 미니 백설기 2개, 개별 모듬팩 3개를 주문하시려고 하시는 거 맞나요?'
                                                                ]})

##### parser

In [ ]:
from pydantic import BaseModel, Field
from datetime import datetime
from typing import List, Dict

# dictionary의 key가 이상하게 나오는 경우가 있어 템플릿 수정
class Order(BaseModel):
    products: List[Dict[str, Dict[str, int]]] = Field(
        description="주문 상품 별 가격과 주문 수량\n예시: [{'상품명': {'가격': 1000, '수량': 2}}, {'상품명': {'가격': 2000, '수량': 3}}]\n모든 dictionary의 key는 예시와 동일해야만 함"
    )
    created_at: str = Field(description="현재 시간: 뒷부분에 표시된 datetime 형태의 문자열") # created_at 필드가 추출이 안 되는데 datetime 타입이 때문인가 해서 str로 변경
    
order_record_parser = PydanticOutputParser(pydantic_object=Order)
order_record_parser.get_format_instructions()

##### prompt

In [ ]:
order_template = """
<이전 대화>를 보고 고객의 주문 내역을 정리해줘.
{format_instructions}

<이전 대화>
{history}
답변:"""

order_record_prompt = PromptTemplate(
   template = order_template,
   input_variables=["history"],
   partial_variables={"format_instructions": order_record_parser.get_format_instructions()},
)


order_query_template = """
<가게에서 판매하는 상품 목록>과 <이전 대화>를 보고 주문 내역과 주문 내역 확인 당시의 '현재 시간' 값을 파악해줘.
{format_instructions}

<가게에서 판매하는 상품 목록>
1. 상품: 떡케익5호
   기본 판매 수량: 1개
   기본 판매 수량의 가격: 54,000원
2. 상품: 무지개 백설기 케익
   기본 판매 수량: 1개
   기본 판매 수량의 가격: 51,500원
3. 상품: 미니 백설기
   기본 판매 수량: 35개
   기본 판매 수량의 가격: 31,500원
4. 상품: 개별 모듬팩
   기본 판매 수량: 1개
   기본 판매 수량의 가격: 13,500원

<이전 대화>
{history}
답변:"""

order_prompt = PromptTemplate(
   template = order_query_template,
   input_variables=["history"],
   partial_variables={"format_instructions": order_record_parser.get_format_instructions()},
)


order_change_template = """
<가게에서 판매하는 상품 목록>과 <이전 대화>를 보고 고객의 주문변경 내역을 정리해줘.
주문변경 파악은 실수하기 쉬운 작업이니 단계를 나누어 꼼꼼히 진행해야 해.
단계1. <이전 대화>를 꼼꼼히 살펴보면서 고객이 주문했던 내용을 파악한다.
단계2. 고객이 원래 주문에서 변경한 내용을 파악한다.
단계3. 변경 내용을 빠짐없이 정리한다. 이때 고객의 주문 변경 내용을 원래 주문했던 내용과 헷갈리지 않도록 주의해야 해.
{format_instructions}

<가게에서 판매하는 상품 목록>
1. 상품: 떡케익5호
   기본 판매 수량: 1개
   기본 판매 수량의 가격: 54,000원
2. 상품: 무지개 백설기 케익
   기본 판매 수량: 1개
   기본 판매 수량의 가격: 51,500원
3. 상품: 미니 백설기
   기본 판매 수량: 35개
   기본 판매 수량의 가격: 31,500원
4. 상품: 개별 모듬팩
   기본 판매 수량: 1개
   기본 판매 수량의 가격: 13,500원

<이전 대화>
{history}
답변:"""

order_change_prompt = PromptTemplate(
   template = order_change_template,
   input_variables=["history"],
   partial_variables={"format_instructions": order_record_parser.get_format_instructions()},
)

##### helper function

In [ ]:
from datetime import datetime

class OrderIDGenerator:
    def __init__(self):
        self.current_id = 0

    def get_new_order_id(self):
        self.current_id += 1
        return self.current_id

order_id_generator = OrderIDGenerator()

def generate_order_data(order_info: Order):
    unique_id = datetime.now().strftime("%Y-%m-%d %H:%M:%S") # user_name으로 바꾸기
    order_id = order_id_generator.get_new_order_id()  # 순차적으로 증가하는 order_id

    return {
        "user_id": unique_id,
        "order_id": order_id,
        "datetime": order_info.created_at, # 이걸 빼먹고 있었음
        "products": order_info.products,
    }

In [ ]:
def process_order(dict):
    order_dict = generate_order_data(dict['parsed_record'])
    task_result = create_order(order_dict)
    return task_result

##### report_chain

In [ ]:
report_prompt = PromptTemplate.from_template("""
<작업 수행 결과>를 바탕으로 고객 요청에 대한 답변을 친절하고 자세하게 알려줘.
고객이 요청한 작업을 파악할 때는 <이전 대화>도 참고해. 

답변에는 크게 3가지 내용이 담겨 있어야 해.
1. 고객이 요청한 작업
2. 고객이 요청한 작업에 따라 파악한 정보
3. 고객이 요청한 작업에 대한 수행 결과

<이전 대화>:
{history}

<고객이 요청한 작업>:
{customer_message}

<고객 요청에 따라 파악한 정보>:
{parsed_record}

<작업 수행 결과>:
{task_result}
답변:""")

report_chain = report_prompt | ChatOpenAI(model="gpt-3.5-turbo-0613") | StrOutputParser()

##### request 별 처리 chain

In [ ]:
order_chain = (
    RunnablePassthrough.assign(parsed_record=order_prompt | ChatOpenAI(model="gpt-3.5-turbo-0613") | order_record_parser) |
    RunnablePassthrough.assign(task_result=RunnableLambda(process_order)) |
    report_chain
)
order_query_chain
order_change_chain
order_cancel_chain

##### router(request_type_branch)

In [ ]:
request_type_branch = RunnableBranch(
  (lambda x: x["request"] == "주문 조회 요청", order_query_chain),
  (lambda x: x["request"] == "주문 변경 요청", order_change_chain),
  (lambda x: x["request"] == "주문 취소 요청", order_cancel_chain),
  order_chain
)